# <center>Oracle to Cloud Spanner

In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#### References

- [DataprocPySparkBatchOp reference](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.0/google_cloud_pipeline_components.experimental.dataproc.html)
- [Kubeflow SDK Overview](https://www.kubeflow.org/docs/components/pipelines/sdk/sdk-overview/)
- [Dataproc Serverless in Vertex AI Pipelines tutorial](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_dataproc_serverless_pipeline_components.ipynb)
- [Build a Vertex AI Pipeline](https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline)

#### Overview - Oracle to Cloud Spanner Migration
This notebook helps with the step by step process of migrating Oracle database tables to Cloud Spanner using Dataproc template. This notebook solution uses the [JDBCTOSPANNER](../../java/src/main/java/com/google/cloud/dataproc/templates/jdbc/README.md) Spark (Java) template. The migration will create the Spanner tables if not exist, and it will overwrite or append based on the write mode configured in the notebook.
##### Feedback
Share you feedback, ideas, thoughts [feedback-form](https://forms.gle/XXCJeWeCJJ9fNLQS6). 
Questions, issues, and comments should be directed to dataproc-templates-support-external@googlegroups.com

#### Permissions
This notebook is built to run a Vertex AI User-Managed Notebook using the default Compute Engine Service Account.  
Check the Dataproc Serverless in Vertex AI Pipelines tutorial linked above to learn how to setup a different Service Account. If using custom service account, service account attached to Vertex AI notebook should have Service Account User role to use custom role in job.

Make sure that the service account used to run the notebook has the following roles:

- roles/aiplatform.serviceAgent
- roles/aiplatform.customCodeServiceAgent
- roles/storage.objectCreator
- roles/storage.objectViewer
- roles/dataproc.editor
- roles/dataproc.worker
- roles/bigquery.dataEditor

## Step 1: Install Libraries
#### Run Step 1 one time for each new notebook instance

In [ ]:
! pip3 install SQLAlchemy
! pip3 install --upgrade google-cloud-pipeline-components kfp --user -q
! pip3 install cx-Oracle

In [ ]:
!sudo apt-get update -y
!sudo apt-get install default-jdk -y
!sudo apt-get install maven -y

#### Oracle client Installation

In [ ]:
%%bash
sudo mkdir -p /opt/oracle
sudo rm -fr /opt/oracle/instantclient*
cd /opt/oracle
sudo wget --no-verbose https://download.oracle.com/otn_software/linux/instantclient/instantclient-basic-linuxx64.zip
sudo unzip instantclient-basic-linuxx64.zip
INSTANT_CLIENT_DIR=$(find /opt/oracle -maxdepth 1 -type d -name "instantclient_[0-9]*_[0-9]*" | sort | tail -1)
test -n "${INSTANT_CLIENT_DIR}" || echo "ERROR: Could not find instant client"
test -n "${INSTANT_CLIENT_DIR}" || exit 1
sudo apt-get install libaio1
sudo sh -c "echo ${INSTANT_CLIENT_DIR} > /etc/ld.so.conf.d/oracle-instantclient.conf"
sudo ldconfig
export LD_LIBRARY_PATH=${INSTANT_CLIENT_DIR}:$LD_LIBRARY_PATH

#### Once you've installed the additional packages, you may need to restart the notebook kernel so it can find the packages.

Uncomment & Run this cell if you have installed anything from above commands

In [ ]:
# import os
# import IPython
# if not os.getenv("IS_TESTING"):
#     app = IPython.Application.instance()
#     app.kernel.do_shutdown(True)

## Step 2: Import Libraries

In [ ]:
import sqlalchemy
import math ,os
import google.cloud.aiplatform as aiplatform
from kfp import dsl
from kfp.v2 import compiler
from datetime import datetime
import time
import copy
import json
import pandas as pd
from pathlib import Path
from google_cloud_pipeline_components.experimental.dataproc import DataprocSparkBatchOp

## Step 3: Assign Parameters

### Step 3.1 Common Parameters
 
- PROJECT : GCP project-id
- REGION : GCP region
- GCS_STAGING_LOCATION : GCS staging location to be used for this notebook to store artifacts
- SUBNET : VPC subnet
- JARS : list of jars. For this notebook oracle connector is required in addition with the dataproc template jars
- MAX_PARALLELISM : Parameter for number of jobs to run in parallel default value is 5
- SERVICE_ACCOUNT : Custom service account email to use for vertex ai pipeline and dataproc job with above mentioned permissions

In [ ]:
PROJECT = "<project-id>"
REGION = "<region>"
GCS_STAGING_LOCATION = "<gs://bucket/[folder]>"
SUBNET = "<projects/{project}/regions/{region}/subnetworks/{subnet}>"
MAX_PARALLELISM = 5 # max number of tables which will migrated parallelly 
SERVICE_ACCOUNT = "<Service-Account>"

# Do not change this parameter unless you want to refer below JARS from new location
JARS = [GCS_STAGING_LOCATION + "/jars/ojdbc8-21.7.0.0.jar"]

In [ ]:
# If SERVICE_ACCOUNT is not specified it will take the one attached to Notebook
if SERVICE_ACCOUNT == "":
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
    print("Service Account: ",SERVICE_ACCOUNT)

### Step 3.2 Oracle to Cloud Spanner Parameters
- ORACLE_HOST : Oracle instance ip address
- ORACLE_PORT : Oracle instance port
- ORACLE_USERNAME : Oracle username
- ORACLE_PASSWORD : Oracle password
- ORACLE_DATABASE : Name of database/service for Oracle connection
- ORACLE_TABLE_LIST : list of tables you want to migrate eg: ['table1','table2'] else provide an empty list for migration whole database eg : []
- SPANNER_INSTANCE : cloud spanner instance name
- SPANNER_DATABASE : cloud spanner database name
- SPANNER_TABLE_PRIMARY_KEYS : provide dictionary of format {"table_name":"primary_key"} for tables which do not have primary key in Oracle
- SPANNER_OUTPUT_MODE: <Append | Overwrite>

In [ ]:
ORACLE_HOST = "<host-ip>"
ORACLE_PORT = "<port>"
ORACLE_USERNAME = "<username>"
ORACLE_PASSWORD = "<password>"
ORACLE_DATABASE = "<database>"
ORACLE_TABLE_LIST = [] # leave list empty for migrating complete database else provide tables as ['table1','table2']

SPANNER_OUTPUT_MODE = "<Overwrite | Append>"
SPANNER_INSTANCE = "<spanner-instance>"
SPANNER_DATABASE = "<spanner-database>"
SPANNER_TABLE_PRIMARY_KEYS = {} # provide table which do not have PK in Oracle {"table_name":"primary_key"}

### Step 3.3 Notebook Configuration Parameters
Below variables should not be changed unless required
- ORACLE_URL : Oracle Python URL
- JDBC_DRIVER : JDBC driver class
- JDBC_URL : Oracle JDBC URL
- JDBC_FETCH_SIZE : Determines how many rows to fetch per round trip
- JDBC_SESSION_INIT_STATEMENT : Custom SQL statement to execute in each reader database session
- MAIN_CLASS : Dataproc Template Main Class
- WORKING_DIRECTORY : Python working directory
- PACKAGE_EGG_FILE : Dataproc Template distributio file
- PIPELINE_ROOT : Path to Vertex AI pipeline artifacts

In [ ]:
cur_path = Path(os.getcwd())
WORKING_DIRECTORY = os.path.join(cur_path.parent.parent ,'python')

# If the above code doesn't fetches the correct path please
# provide complete path to python folder in your dataproc 
# template repo which you cloned 

# WORKING_DIRECTORY = "/home/jupyter/dataproc-templates/python/"
print(WORKING_DIRECTORY)

In [ ]:
ORACLE_URL = "oracle://{}:{}@{}:{}?service_name={}".format(ORACLE_USERNAME,ORACLE_PASSWORD,ORACLE_HOST,ORACLE_PORT,ORACLE_DATABASE)
JDBC_DRIVER = "oracle.jdbc.OracleDriver"
JDBC_URL = "jdbc:oracle:thin:{}/{}@{}:{}/{}".format(ORACLE_USERNAME,ORACLE_PASSWORD,ORACLE_HOST,ORACLE_PORT,ORACLE_DATABASE)
JDBC_FETCH_SIZE = 200
JDBC_SESSION_INIT_STATEMENT = "BEGIN DBMS_APPLICATION_INFO.SET_MODULE('Dataproc Templates','OracleToSpanner Notebook'); END;"
MAIN_CLASS = "com.google.cloud.dataproc.templates.main.DataProcTemplate"
JAR_FILE = "dataproc-templates-1.0-SNAPSHOT.jar"
GRPC_JAR_PATH = "./grpc_lb/io/grpc/grpc-grpclb/1.40.1"
GRPC_JAR = "grpc-grpclb-1.40.1.jar"
LOG4J_PROPERTIES_PATH = "./src/test/resources"
LOG4J_PROPERTIES = "log4j-spark-driver-template.properties"
PIPELINE_ROOT = GCS_STAGING_LOCATION + "/pipeline_root/dataproc_pyspark"

# adding dataproc template JAR and grpc jar
JARS.append(GCS_STAGING_LOCATION + "/" + GRPC_JAR)
JARS.append(GCS_STAGING_LOCATION + "/" + JAR_FILE)

## Step 4: Generate ORACLE Table List
This step creates list of tables for migration. If ORACLE_TABLE_LIST is kept empty all the tables in the ORACLE_DATABASE are listed for migration otherwise the provided list is used

In [ ]:
if len(ORACLE_TABLE_LIST) == 0:
    DB = sqlalchemy.create_engine(ORACLE_URL)
    with DB.connect() as conn:
        print("connected to database")
        results = DB.execute('SELECT table_name FROM user_tables').fetchall()
        print("Total Tables = ", len(results))
        for row in results:
            ORACLE_TABLE_LIST.append(row[0])

print("list of tables for migration :")
print(ORACLE_TABLE_LIST)

## Step 5: Get Primary Keys for tables not present in SPANNER_TABLE_PRIMARY_KEYS
For tables which do not have primary key provided in dictionary SPANNER_TABLE_PRIMARY_KEYS this step fetches primary key from ORACLE_DATABASE

In [ ]:
DB = sqlalchemy.create_engine(ORACLE_URL)
with DB.connect() as conn:
    for table in ORACLE_TABLE_LIST:
        primary_keys = []
        if table not in SPANNER_TABLE_PRIMARY_KEYS:
            results = DB.execute("SELECT cols.column_name FROM sys.all_constraints cons, sys.all_cons_columns cols WHERE cols.table_name = '{}' AND cons.constraint_type = 'P' AND cons.status = 'ENABLED' AND cons.constraint_name = cols.constraint_name AND cons.owner = cols.owner".format(table)).fetchall()
            for row in results:
                primary_keys.append(row[0])
            if primary_keys:
                SPANNER_TABLE_PRIMARY_KEYS[table] = ",".join(primary_keys)
            else:
                SPANNER_TABLE_PRIMARY_KEYS[table] = ""

In [ ]:
pkDF = pd.DataFrame({"table" : ORACLE_TABLE_LIST, "primary_keys": list(SPANNER_TABLE_PRIMARY_KEYS.values())})
print("Below are identified primary keys for migrating oracle table to spanner:")
pkDF

# Step 6: Get Row Count of Tables and identify Partition Columns
This step uses PARTITION_THRESHOLD(default value is 1 million) parameter and any table having rows greater than PARTITION_THRESHOLD will be used for partitioned read based on Primary Keys
 - CHECK_PARTITION_COLUMN_LIST : List will have table and its partitioned column if exceeds threshold

In [ ]:
PARTITION_THRESHOLD = 1000000
CHECK_PARTITION_COLUMN_LIST={}

In [ ]:
with DB.connect() as conn:
    for table in ORACLE_TABLE_LIST:
        results = DB.execute("SELECT count(1) FROM {}".format(table)).fetchall()
        if results[0][0]>int(PARTITION_THRESHOLD) and len(SPANNER_TABLE_PRIMARY_KEYS.get(table).split(",")[0])>0:
            column_list=SPANNER_TABLE_PRIMARY_KEYS.get(table).split(",")
            column = column_list[0]
            results_datatype = DB.execute("select DATA_TYPE from sys.all_tab_columns where TABLE_NAME = '{0}' and COLUMN_NAME = '{1}'".format(table,column)).fetchall()      
            if results_datatype[0][0]=="NUMBER" or results_datatype[0][0]=="INTEGER":
                lowerbound = DB.execute("SELECT min({0}) from {1}".format(column,table)).fetchall()
                upperbound = DB.execute("SELECT max({0}) from {1}".format(column,table)).fetchall()
                numberPartitions = math.ceil((upperbound[0][0]-lowerbound[0][0])/PARTITION_THRESHOLD)
                CHECK_PARTITION_COLUMN_LIST[table]=[column,lowerbound[0][0],upperbound[0][0],numberPartitions]

print("Below dictionary represents {table_name : [primary_key, lowerbound, upperbound, numberPartitions]}")
print(CHECK_PARTITION_COLUMN_LIST)

## Step 7: Download JAR files and Upload to GCS (only required to run one-time)
#### Run Step 7 one time for each new notebook instance

In [ ]:
%cd $WORKING_DIRECTORY

#### Setting PATH variables for JDK and Maven and executing MAVEN build

In [ ]:
!wget --no-verbose https://repo1.maven.org/maven2/com/oracle/database/jdbc/ojdbc8/21.7.0.0/ojdbc8-21.7.0.0.jar
!mvn -q clean spotless:apply install -DskipTests 
!mvn -q dependency:get -Dartifact=io.grpc:grpc-grpclb:1.40.1 -Dmaven.repo.local=./grpc_lb 

#### Copying JARS files to GCS_STAGING_LOCATION

In [ ]:
!gsutil cp target/$JAR_FILE $GCS_STAGING_LOCATION/$JAR_FILE
!gsutil cp $GRPC_JAR_PATH/$GRPC_JAR $GCS_STAGING_LOCATION/$GRPC_JAR
!gsutil cp $LOG4J_PROPERTIES_PATH/$LOG4J_PROPERTIES $GCS_STAGING_LOCATION/$LOG4J_PROPERTIES
!gsutil cp ojdbc8-21.7.0.0.jar $GCS_STAGING_LOCATION/jars/ojdbc8-21.7.0.0.jar

## Step 8: Calculate Parallel Jobs for Oracle to Cloud Spanner
This step uses MAX_PARALLELISM parameter to calculate number of parallel jobs to run

In [ ]:
# calculate parallel jobs:
COMPLETE_LIST = copy.deepcopy(ORACLE_TABLE_LIST)
PARALLEL_JOBS = len(ORACLE_TABLE_LIST)//MAX_PARALLELISM
JOB_LIST = []
while len(COMPLETE_LIST) > 0:
    SUB_LIST = []
    for i in range(MAX_PARALLELISM):
        if len(COMPLETE_LIST)>0 :
            SUB_LIST.append(COMPLETE_LIST[0])
            COMPLETE_LIST.pop(0)
        else:
            break
    JOB_LIST.append(SUB_LIST)
print("list of tables for execution : ")
print(JOB_LIST)

## Step 9: Execute Pipeline to Migrate tables from Oracle to Cloud Spanner

In [ ]:
oracle_to_spanner_jobs = []

In [ ]:
def migrate_oracle_to_spanner(EXECUTION_LIST):
    EXECUTION_LIST = EXECUTION_LIST
    aiplatform.init(project=PROJECT,staging_bucket=GCS_STAGING_LOCATION)
    
    @dsl.pipeline(
        name="java-oracle-to-spanner-spark",
        description="Pipeline to get data from Oracle to Cloud Spanner",
    )
    def pipeline(
        PROJECT_ID: str = PROJECT,
        LOCATION: str = REGION,
        MAIN_CLASS: str = MAIN_CLASS,
        JAR_FILE_URIS: list = JARS,
        SUBNETWORK_URI: str = SUBNET,
        FILE_URIS: list = [GCS_STAGING_LOCATION + "/" + LOG4J_PROPERTIES]
    ):
        for table in EXECUTION_LIST:
            BATCH_ID = "ora2spanner-{}-{}".format(table,datetime.now().strftime("%s")).replace('_','-').lower()
            oracle_to_spanner_jobs.append(BATCH_ID)
            if table in CHECK_PARTITION_COLUMN_LIST.keys():
                TEMPLATE_SPARK_ARGS = [
                "--template=JDBCTOSPANNER",
                "--templateProperty", "project.id={}".format(PROJECT),
                "--templateProperty", "jdbctospanner.jdbc.url={}".format(JDBC_URL),
                "--templateProperty", "jdbctospanner.jdbc.driver.class.name={}".format(JDBC_DRIVER),
                "--templateProperty", "jdbctospanner.jdbc.fetchsize={}".format(JDBC_FETCH_SIZE),
                "--templateProperty", "jdbctospanner.sql=select * from {}".format(table),
                "--templateProperty", "jdbctospanner.output.instance={}".format(SPANNER_INSTANCE),
                "--templateProperty", "jdbctospanner.output.database={}".format(SPANNER_DATABASE),
                "--templateProperty", "jdbctospanner.output.table={}".format(table),
                "--templateProperty", "jdbctospanner.output.saveMode={}".format(SPANNER_OUTPUT_MODE),
                "--templateProperty", "jdbctospanner.output.primaryKey={}".format(SPANNER_TABLE_PRIMARY_KEYS[table]),
                "--templateProperty", "jdbctospanner.output.batchInsertSize=200",
                "--templateProperty", "jdbctospanner.sql.partitionColumn={}".format(CHECK_PARTITION_COLUMN_LIST[table][0]),
                "--templateProperty", "jdbctospanner.sql.lowerBound={}".format(CHECK_PARTITION_COLUMN_LIST[table][1]),
                "--templateProperty", "jdbctospanner.sql.upperBound={}".format(CHECK_PARTITION_COLUMN_LIST[table][2]),
                "--templateProperty", "jdbctospanner.sql.numPartitions={}".format(CHECK_PARTITION_COLUMN_LIST[table][3]),
                ]
            else:
                TEMPLATE_SPARK_ARGS = [
                "--template=JDBCTOSPANNER",
                "--templateProperty", "project.id={}".format(PROJECT),
                "--templateProperty", "jdbctospanner.jdbc.url={}".format(JDBC_URL),
                "--templateProperty", "jdbctospanner.jdbc.driver.class.name={}".format(JDBC_DRIVER),
                "--templateProperty", "jdbctospanner.jdbc.fetchsize={}".format(JDBC_FETCH_SIZE),
                "--templateProperty", "jdbctospanner.sql=select * from {}".format(table),
                "--templateProperty", "jdbctospanner.output.instance={}".format(SPANNER_INSTANCE),
                "--templateProperty", "jdbctospanner.output.database={}".format(SPANNER_DATABASE),
                "--templateProperty", "jdbctospanner.output.table={}".format(table),
                "--templateProperty", "jdbctospanner.output.saveMode={}".format(SPANNER_OUTPUT_MODE),
                "--templateProperty", "jdbctospanner.output.primaryKey={}".format(SPANNER_TABLE_PRIMARY_KEYS[table]),
                "--templateProperty", "jdbctospanner.output.batchInsertSize=200",
                ]
            if JDBC_SESSION_INIT_STATEMENT:
                TEMPLATE_SPARK_ARGS.append("--jdbctospanner.input.sessioninitstatement={}".format(JDBC_SESSION_INIT_STATEMENT))
    
            _ = DataprocSparkBatchOp(
                project=PROJECT_ID,
                location=LOCATION,
                batch_id=BATCH_ID,
                main_class=MAIN_CLASS,
                jar_file_uris=JAR_FILE_URIS,
                file_uris=FILE_URIS,
                subnetwork_uri=SUBNETWORK_URI,
                runtime_config_version="1.1", # issue 665
                args=TEMPLATE_SPARK_ARGS
            )
            time.sleep(3)

    compiler.Compiler().compile(pipeline_func=pipeline, package_path="pipeline.json")

    pipeline = aiplatform.PipelineJob(
        display_name="pipeline",
        template_path="pipeline.json",
        pipeline_root=PIPELINE_ROOT,
        enable_caching=False,
    )
    pipeline.run()

In [ ]:
for execution_list in JOB_LIST:
    print(execution_list)
    migrate_oracle_to_spanner(execution_list)

## Step 10: Get status for tables migrated from Oracle to Cloud Spanner

In [ ]:
def get_bearer_token():
    
    try:
        #Defining Scope
        CREDENTIAL_SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]

        #Assining credentials and project value
        credentials, project_id = google.auth.default(scopes=CREDENTIAL_SCOPES)

        #Refreshing credentials data
        credentials.refresh(requests.Request())

        #Get refreshed token
        token = credentials.token
        if token:
            return (token,200)
        else:
            return "Bearer token not generated"
    except Exception as error:
        return ("Bearer token not generated. Error : {}".format(error),500)

In [ ]:
from google.auth.transport import requests
import google
token = get_bearer_token()
if token[1] == 200:
    print("Bearer token generated")
else:
    print(token)

In [ ]:
import requests

oracle_to_spanner_status = []
job_status_url = "https://dataproc.googleapis.com/v1/projects/{}/locations/{}/batches/{}"
for job in oracle_to_spanner_jobs:
    auth = "Bearer " + token[0]
    url = job_status_url.format(PROJECT,REGION,job)
    headers = {
      'Content-Type': 'application/json; charset=UTF-8',
      'Authorization': auth 
    }
    response = requests.get(url, headers=headers)
    oracle_to_spanner_status.append(response.json()['state'])

In [ ]:
statusDF = pd.DataFrame({"table" : ORACLE_TABLE_LIST,"oracle_to_spanner_job" : oracle_to_spanner_jobs, "oracle_to_spanner_status" : oracle_to_spanner_status})
statusDF

## Step 11: Validate row counts of migrated tables from Oracle to Cloud Spanner

In [ ]:
oracle_row_count = []
spanner_row_count = []

In [ ]:
# get Oracle table counts
DB = sqlalchemy.create_engine(ORACLE_URL)
with DB.connect() as conn:
    for table in ORACLE_TABLE_LIST:
        results = DB.execute("select count(*) from {}".format(table)).fetchall()
        for row in results:
            oracle_row_count.append(row[0])

In [ ]:
# get spanner table counts
from google.cloud import spanner

spanner_client = spanner.Client()
instance = spanner_client.instance(SPANNER_INSTANCE)
database = instance.database(SPANNER_DATABASE)

for table in ORACLE_TABLE_LIST:
    with database.snapshot() as snapshot:
        results = snapshot.execute_sql("select count(*) from {}".format(table))
        for row in results:
            spanner_row_count.append(row[0])

In [ ]:
statusDF['oracle_row_count'] = oracle_row_count 
statusDF['spanner_row_count'] = spanner_row_count 
statusDF